In [1]:
import numpy as np
import sys
if "/opt/ros/kinetic/lib/python2.7/dist-packages" in sys.path:
    sys.path.remove("/opt/ros/kinetic/lib/python2.7/dist-packages")
import cv2
import struct
import math
import time

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import os 

from network.textnet import TextNet
from util.detection import TextDetector
from util.augmentation import BaseTransform
from util.config import config as cfg, update_config, print_config
from util.option import BaseOptions
from util.visualize import visualize_detection
from util.misc import to_device, mkdirs, rescale_result
from rotate_input import rotate_cv, rotate_back, rotate_back_change_h_w
from PIL import Image as Im
import tools.utils as utils
import tools.dataset as dataset
from models.moran import MORAN
from collections import OrderedDict

import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
print(sys.version)

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


In [3]:
def read_commodity(path):
    _list = []
    for line in open(path, "r"):
        line = line.rstrip('\n')
        _list.append(line)
    print ("Finish reading list")
    return _list


In [4]:
alphabet = '0:1:2:3:4:5:6:7:8:9:a:b:c:d:e:f:g:h:i:j:k:l:m:n:o:p:q:r:s:t:u:v:w:x:y:z:$' 
cuda_use = torch.cuda.is_available()
means = (0.485, 0.456, 0.406)
stds = (0.229, 0.224, 0.225)
bbox_thres = 1500
color_map = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,255,255)] # 0 90 180 270 noise
commodity_list = read_commodity("./config/commodity_list.txt")

csv_file = "./mask/val.csv"
data_list = pd.read_csv(csv_file)

Finish reading list


In [5]:
snake_network = TextNet(is_training=False, backbone='vgg')
if cuda_use:
    snake_network = snake_network.cuda()
    cuda_flag = True
    MORAN_network = MORAN(1, len(alphabet.split(':')), 256, 32, 100, BidirDecoder=True, CUDA=cuda_flag)
    MORAN_network = MORAN_network.cuda()
    MORAN_state_dict = torch.load(os.path.join("./model/", "moran.pth"))
else:
    MORAN_network = MORAN(1, len(alphabet.split(':')), 256, 32, 100, BidirDecoder=True, inputDataType='torch.FloatTensor', CUDA=cuda_flag)
    MORAN_state_dict = torch.load(os.path.join("./model/", "moran.pth"), map_location='cpu')
MORAN_state_dict_rename = OrderedDict()
for k, v in MORAN_state_dict.items():
    name = k.replace("module.", "") # remove `module.`
    MORAN_state_dict_rename[name] = v
    
MORAN_network.load_state_dict(MORAN_state_dict_rename)
converter = utils.strLabelConverterForAttention(alphabet, ':')
transformer = dataset.resizeNormalize((100, 32))

for p in MORAN_network.parameters():
    p.requires_grad = False
snake_network.load_model(os.path.join("./model/", "textsnake_vgg_180.pth"))   
MORAN_network.eval()
snake_network.eval()
detector = TextDetector(snake_network, tr_thresh=0.6, tcl_thresh=0.4)


/home/andyser/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/andyser/code/thesis/semantic_labels_sys/thesis/benchmark/models/asrn_res.py:234: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_out', a=0)
/home/andyser/code/thesis/semantic_labels_sys/thesis/benchmark/models/asrn_res.py:236: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.weight, 1)
/home/andyser/code/thesis/semantic_labels_sys/thesis/benchmark/models/asrn_res.py:237: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias, 0)


Loading from ./model/textsnake_vgg_180.pth


In [6]:
def region_predict(img):
    # # Preprocessing
    (rows, cols, channels) = img.shape
    image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    torch.cuda.synchronize()
    start = time.time()

    x = image.astype(np.float32)
    x = (x / 255 - means) / stds
    x = x.astype(np.float32)
    x = x[:, :, ::-1].copy()
    x = torch.from_numpy(x).permute(2, 0, 1)
    x = Variable(x.unsqueeze(0)) 
    if cuda_use:
        x = x.cuda()
    contours, output = detector.detect(x)

    torch.cuda.synchronize()
    end = time.time()
#     print ("Text Detection Time : {}".format(end - start))

    image, contours = rescale_result(image, contours, rows, cols)
    img_viz = visualize_detection(image, contours)

    return img_viz, contours

## bbox = [xmin, xmax, ymin, ymax]
def recog_predict(bbox, contour, img, img_vis, mask, rot=0):
    # # Preprocessing
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (rows, cols, channels) = img.shape
    
    if (bbox[3] - bbox[2]) * (bbox[1] - bbox[0]) < bbox_thres:
        return img, mask
    start = time.time()
    image = gray[bbox[2]:bbox[3], bbox[0]:bbox[1]]
    
#     plt.figure()
#     plt.imshow(image)
#     plt.show()
    
    image = Im.fromarray(image) 
    image = transformer(image)

    if cuda_use:
        image = image.cuda()
    image = image.view(1, *image.size())
    image = Variable(image)
    text = torch.LongTensor(1 * 5)
    length = torch.IntTensor(1)
    text = Variable(text)
    length = Variable(length)

    max_iter = 20
    t, l = converter.encode('0'*max_iter)
    utils.loadData(text, t)
    utils.loadData(length, l)
    output = MORAN_network(image, length, text, text, test=True, debug=True)

    preds, preds_reverse = output[0]
    demo = output[1]

    _, preds = preds.max(1)
    _, preds_reverse = preds_reverse.max(1)

    sim_preds = converter.decode(preds.data, length.data)
    sim_preds = sim_preds.strip().split('$')[0]
    sim_preds_reverse = converter.decode(preds_reverse.data, length.data)
    sim_preds_reverse = sim_preds_reverse.strip().split('$')[0]
#     print (sim_preds)
    # print('\nResult:\n' + 'Left to Right: ' + sim_preds + '\nRight to Left: ' + sim_preds_reverse + '\n\n')
#     print ("Text Recognize Time : {}".format(time.time() - start))

    if sim_preds in commodity_list:
        cv2.rectangle(img_vis, (bbox[0], bbox[2]),(bbox[1], bbox[3]),  (0,0,255), 3)
        cv2.putText(img_vis, sim_preds, (bbox[0], bbox[2]), 0, 1, (0,0,255),3)
        cv2.fillConvexPoly(mask, contour, commodity_list.index(sim_preds) + rot*len(commodity_list) + 1)
    else:
        correct, conf, _bool = conf_of_word(sim_preds)

        if _bool:
            cv2.putText(img_vis, correct + "{:.2f}".format(conf), (bbox[0], bbox[2]), 0, 1, (255,0,0),3)
            cv2.rectangle(img_vis, (bbox[0], bbox[2]),(bbox[1], bbox[3]), (255,0,0), 2)
            cv2.fillConvexPoly(mask, _cont, commodity_list.index(correct) + rot*len(commodity_list) + 1)
        else:
            cv2.putText(img_vis, sim_preds, (bbox[0], bbox[2]), 0, 1, (0, 0, 0),3)
            cv2.rectangle(img_vis, (bbox[0], bbox[2]),(bbox[1], bbox[3]), (0, 0, 0), 2)
        print (sim_preds, conf, correct, rot*90)

    return img, mask

In [7]:
def conf_of_word(target):
    ### Edit distance
    total = np.zeros(len(commodity_list))
    for i in range(1, len(commodity_list)):
        size_x = len(commodity_list[i]) + 1
        size_y = len(target) + 1
        matrix = np.zeros ((size_x, size_y))
        for x in range(size_x):
            matrix [x, 0] = x
        for y in range(size_y):
            matrix [0, y] = y

        for x in range(1, size_x):
            for y in range(1, size_y):
                if commodity_list[i][x-1] == target[y-1]:
                    matrix [x,y] = min(
                        matrix[x-1, y] + 1,
                        matrix[x-1, y-1],
                        matrix[x, y-1] + 1
                    )
                else:
                    matrix [x,y] = min(
                        matrix[x-1,y] + 1,
                        matrix[x-1,y-1] + 1,
                        matrix[x,y-1] + 1
                    )
        # print (matrix)
        total[i] = (size_x - matrix[size_x-1, size_y-1]) / float(size_x)
    return commodity_list[np.argmax(total)], np.max(total), np.max(total) >= 0.77   ## 0.77

In [8]:
def adj(_img, _level = 8):
    (colomn, row) = _img.shape
    _count = 0
    _pixel_pair = []
    label = np.zeros((colomn,row),dtype = np.uint8)
    for i in range(colomn):
        for j in range(row):
            if (_img[i,j] == 1 and label[i,j] == 0):
                _pixel_pair.append([i,j])
                _count += 1
            while len(_pixel_pair) != 0:
                pair = _pixel_pair.pop()
                a = pair[1] + 1
                b = pair[1] - 1
                c = pair[0] + 1
                d = pair[0] - 1
                if a == row : a -= 1
                if b == -1  : b += 1
                if c == colomn : c -= 1
                if d == -1  : d += 1

                if _img[pair[0],a] == 1 and label[pair[0],a] == 0:
                    _pixel_pair.append([pair[0],a])
                if _img[pair[0],b] == 1 and label[pair[0],b] == 0:
                    _pixel_pair.append([pair[0],b])
                if _img[c,pair[1]] == 1 and label[c,pair[1]] == 0:
                    _pixel_pair.append([c,pair[1]])
                if _img[d,pair[1]] == 1 and label[d,pair[1]] == 0:
                    _pixel_pair.append([d,pair[1]])
                if _level == 8:
                    if _img[c,a] == 1 and label[c,a] == 0:
                        _pixel_pair.append([c,a])
                    if _img[d,a] == 1 and label[d,a] == 0:
                        _pixel_pair.append([d,a])
                    if _img[d,b] == 1 and label[d,b] == 0:
                        _pixel_pair.append([d,b])
                    if _img[c,b] == 1 and label[c,b] == 0:
                        _pixel_pair.append([c,b])
                label[pair[0],pair[1]] = _count

#     print("Num of classes for connected components : ", _count)
    return label

In [30]:
dataset_name = "text-pnp"
if dataset_name == "text-pnp":
    obj_num = 10
    div = 6
elif dataset_name == "resolition":
    obj_num = 13
    div = 4    
TP = np.zeros((obj_num*4+1), dtype = np.float128)
FP = np.zeros((obj_num*4+1), dtype = np.float128)
FN = np.zeros((obj_num*4+1), dtype = np.float128)
_sum_origin = 0
_sum_detection = 0
_false_nage = 0
theta_sum = 0.
theta_count = 0
iou_sum = 0.
_total_bn_up = 0
for idx in range(len(data_list)):
    img_name   = data_list.iloc[idx, 0]
    img        = cv2.imread(img_name,cv2.IMREAD_COLOR)
    label_name = data_list.iloc[idx, 1]
    label      = cv2.imread(label_name, cv2.IMREAD_GRAYSCALE)
    if "7_obj" not in img_name: # or "color_231" not in img_name:
        continue
#     img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    
    (rows, cols, channels) = img.shape
    rows = int(np.ceil(rows/32.)*32)
    cols = int(np.ceil(cols/32.)*32)
    cv_image1 = np.zeros((rows, cols, channels),dtype = np.uint8)
    label_temp = np.zeros((rows, cols),dtype = np.uint8)
    cv_image1[:img.shape[0],:img.shape[1],:img.shape[2]] = img[:,:,:]
    label_temp[:img.shape[0],:img.shape[1]] = label[:,:]
    img = cv_image1.copy()    
    label = label_temp.copy()
    
    ### Detection & Recognition
    label[label == 246] = 0   #### barcode
    
    mask = np.zeros([rows, cols], dtype = np.uint8)
    mask_region = np.zeros([rows, cols], dtype = np.uint8)
    img_list_0_90_180_270 = rotate_cv(img)
    
    for i in range(4):

        predict_img, contours = region_predict(img_list_0_90_180_270[i])
        img_bbox = img_list_0_90_180_270[i].copy()
      
        img_input_vis = img_bbox.copy()
        img_input_rec = img_bbox.copy()
        (_rows, _cols, _channels) = img_bbox.shape
        mask_vis = np.zeros([_rows, _cols], dtype = np.uint8)
        mask_vis_temp = np.zeros([_rows, _cols], dtype = np.uint8)
        
        for _cont in contours:
            
            cv2.fillConvexPoly(mask_vis_temp, _cont, 1)
            
            cv2.drawContours(predict_img, [_cont], -1, color_map[i], 3)
            cv2.rectangle(img_bbox, (min(_cont[:,0]), min(_cont[:,1])),(max(_cont[:,0]), max(_cont[:,1])), color_map[i], 3)
            bbox = [min(_cont[:,0]),max(_cont[:,0]),min(_cont[:,1]),max(_cont[:,1])]
            img_input_vis, mask_vis = recog_predict(bbox, _cont, img_input_rec,img_input_vis, mask_vis, rot = i)
            img_input_vis = cv2.cvtColor(img_input_vis, cv2.COLOR_BGR2RGB)
#             cv2.imwrite("./123/bb_img_{}.jpg".format(90*i), img_input_vis)
            
        if i == 0:
            pass
        elif i == 1:
            mask_vis = rotate_back_change_h_w(mask_vis, angle = -90)
            mask_vis_temp = rotate_back_change_h_w(mask_vis_temp, angle = -90)
        elif i == 2:
            mask_vis = rotate_back(mask_vis, angle = -180)
            mask_vis_temp = rotate_back(mask_vis_temp, angle = -180)
        else:
            mask_vis = rotate_back_change_h_w(mask_vis, angle = -270)
            mask_vis_temp = rotate_back_change_h_w(mask_vis_temp, angle = -270)
        mask[mask_vis != 0] = mask_vis[mask_vis != 0]
        mask_region[mask_vis_temp != 0] = mask_vis_temp[mask_vis_temp != 0]
    
    
    ###########
    #### For detection evaluation
    
    mask_region = label*mask_region
    mask_region[mask_region != 0] = 1
    temp_label = label.copy()
    temp_label[temp_label != 0] = 1
    kernel = np.ones((3,3),np.uint8) 
    temp_label = cv2.erode(temp_label,kernel,iterations = 5)
    mask_region = cv2.erode(mask_region,kernel,iterations = 5)  
    temp_label = cv2.dilate(temp_label,kernel,iterations = 5)  
    mask_region = cv2.dilate(mask_region,kernel,iterations = 5)  
    _label_1 = adj(temp_label)
    _label_2 = adj(mask_region)
    _sum_origin += len(np.unique(_label_1)) - 1
    _sum_detection += len(np.unique(_label_2)) - 1 
    
    print (len(np.unique(_label_1)))
    print (len(np.unique(_label_2)))
    
    #### Confusion Matrix
    
    
#     plt.figure()
#     plt.imshow(temp_label)
#     plt.show()
    
#     plt.figure()
#     plt.imshow(mask_region,  cmap = "gray", vmin = 0, vmax = 10)
#     plt.show()

#     plt.figure()
#     plt.imshow(_label_1,  cmap = "gray", vmin = 0, vmax = 10)
#     plt.show()
#     plt.figure()
#     plt.imshow(_label_2,  cmap = "gray", vmin = 0, vmax = 10)
#     plt.show()
    
    label = label/div
    for j in range(obj_num + 1,obj_num*4 + 1):
        c = j
        while(c>obj_num):
            c-=obj_num
        label[label == j] = c
        mask[mask == j] = c
        
    #### Brandbase evaluation
    obj_list = np.unique(label)
    for pix in obj_list:
        if pix != 0:
            mask_source = np.zeros([rows, cols], dtype = np.uint8)
            mask_target = np.zeros([rows, cols], dtype = np.uint8)
            
            mask_source[label == pix] = 1
            mask_target[mask == pix] = 1
            
            label_sor = adj(mask_source)
            label_tar = adj(mask_target)
            pix_sor = np.unique(label_sor)
            pix_tar = np.unique(label_tar)
            for p1 in pix_sor:
                if p1 != 0:
                    _check = False
                    _total_bn_up += 1
                    temp_sor = np.zeros([rows, cols], dtype = np.uint8)
                    temp_sor[label_sor == p1] = 1
                    for p2 in pix_tar: 
                        if p2 != 0:
                            temp_tar = np.zeros([rows, cols], dtype = np.uint8)
                            temp_tar[label_tar == p2] = 1
                            
                            kernel = np.ones((3,3),np.uint8)  
                            
                            # temp_tar = cv2.erode(temp_tar,kernel,iterations = 1)
                            
                            _and_pix = float(sum(sum(np.logical_and(temp_sor, temp_tar))))
                            _or_pix = float(sum(sum(np.logical_or(temp_sor, temp_tar))))
                            iou_rate = _and_pix/_or_pix
                            if iou_rate >= 0.5:
                                contours1, _ = cv2.findContours(temp_sor, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                                contours2, _ = cv2.findContours(temp_tar, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                                
                                rect1 = cv2.minAreaRect(contours1[0])
                                box1 = cv2.boxPoints(rect1)
                                box1 = np.int0(box1)
                                
                                rect2 = cv2.minAreaRect(contours2[0])
                                box2 = cv2.boxPoints(rect2)
                                box2 = np.int0(box2)  
                                
                                theta1 = np.arctan2(box1[0][1]-box1[1][1], box1[0][0]-box1[1][0])/3.1415926*180
                                theta2 = np.arctan2(box2[0][1]-box2[1][1], box2[0][0]-box2[1][0])/3.1415926*180
                                
                                dis = np.absolute(theta2-theta1)
                                if dis > 45:
                                    dis = 90 - dis 
                                
                                theta_count += 1
                                theta_sum += dis
                                iou_sum += iou_rate
                            
                                print (iou_rate,_and_pix,_or_pix)
                                _check = True
                    if not _check:
                        _false_nage += 1
                            
                    
        
    #####
    
    print (np.unique(mask))
    print (np.unique(label))
    target = mask.reshape(rows * cols)
    pred = label.reshape(rows * cols)

    con_matrix = confusion_matrix(target, pred,labels = np.arange(0,obj_num*4+1,1))
    con_matrix[0][0] = 0
    for i in range(0, obj_num*4+1):
        for j in range(0, obj_num*4+1):
            if i == j:
                TP[i] += con_matrix[i][j]
            if i != j:
                FP[j] += con_matrix[i][j]
                FN[i] += con_matrix[i][j]
    print(idx, img_name)
    

sunmaid 0.16666666666666666 hunts 0
anwans 0.5 raisins 0
use 0.3333333333333333 andes 0
brings 0.375 raisins 90
sun 0.5 hunts 90
misdiogram 0.0 crayons 90
which 0.42857142857142855 vanish 90
nonoughty 0.0 crayons 90
university 0.125 raisins 90
messageid 0.0 crayons 90
f 0.3333333333333333 3m 180
anticle 0.2857142857142857 vanish 180
sunmaid 0.16666666666666666 hunts 180
mon 0.42857142857142855 kloone 180
anwns 0.6666666666666666 andes 180
snising 0.625 raisins 180
mass 0.5 raisins 270
kooner 0.7142857142857143 kloone 270
only 0.5 pocky 270
wholepreck 0.0 crayons 270
hunt 0.8333333333333334 hunts 270
5
5
0.7012439384355893 3326.0 4743.0
0.75923682616596 2507.0 3302.0
[0 5 9]
[0. 2. 5. 6. 9.]
0 ./img/7_obj/scene_000006/color_339.jpg
flashstead 0.0 crayons 90
mential 0.2857142857142857 vanish 90
unrealization 0.0 crayons 90
vanishi 0.8571428571428571 vanish 90
we 0.3333333333333333 andes 270
teletrams 0.0 crayons 270
3
3
0.5125013768036127 4653.0 9079.0
0.6620879120879121 2410.0 3640.0
[ 

amwans 0.5 raisins 180
study 0.4 stax 180
slunk 0.42857142857142855 kloone 270
papercloud 0.0 crayons 270
newsgroups 0.0 crayons 270
units 0.6666666666666666 hunts 270
3
3
0.7059124648067571 2758.0 3907.0
[0 9]
[0. 2. 9.]
18 ./img/7_obj/scene_000001/color_296.jpg
comments 0.25 raisins 0
yusiuda 0.25 raisins 180
chirpocted 0.0 crayons 180
computer 0.0 crayons 270
2
2
0.5023088185606487 4460.0 8879.0
[0 3]
[0. 3.]
19 ./img/7_obj/scene_000002/color_303.jpg


In [31]:
print (_total_bn_up)
print (theta_count)
print (theta_sum / theta_count)
print (iou_sum / theta_count)

59
40
2.867427374819038
0.6431336016264122


In [32]:
print (_sum_origin, _sum_detection, _false_nage)
FN[0] = 0
FP[0] = 0
print (TP)
print (FN)
print (FP)

59 53 19
[    0.  3760.     0. 13145. 15061. 14367.  2050.  9277. 12415. 23208.
 14585.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.]
[    0.    26.     0. 11955.  8390.  5967.  1305.  5921. 11002.  9256.
  9715.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.]
[    0.   834. 14457. 32818.    79.   270.  1755.  1186.  6109.   658.
  2314.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.]


In [33]:
recall = TP / (TP + FN)
precision = TP / (TP + FP)
ious = TP / (TP + FN + FP)
fscore = 2*TP / (2*TP + FN + FP)

/home/andyser/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
/home/andyser/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/home/andyser/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/andyser/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [34]:
print (ious)
print (fscore)
print (sum(ious[1:14]/13))
print (sum(fscore[1:14]/13))
2*sum(TP) / (2*sum(TP) + sum(FN) + sum(FP))

[       nan 0.81385281 0.         0.2269588  0.6400765  0.69729179
 0.40117417 0.56622314 0.42047687 0.70068233 0.54801984        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan]
[       nan 0.8973747  0.         0.36995342 0.78054469 0.82165223
 0.5726257  0.72304275 0.59202213 0.82400142 0.70802689        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan]
nan
nan


0.6349789405833061077

In [131]:
### IOU
0.42548957096516391334
0.5284407007142797736          0.5421415715311996553
0.5044586460943159716          0.5388207442850785732
0.38584792222682856477         0.43220951477705407327
0.46073640775482894497         0.42075081174636323325
0.48503068403625244052          0.49718318712174729647

0.4971831871217473

In [53]:
### F-score
0.59014847078778318024        0.6139501681337725087
0.6852427911485629876
0.66380536213827919965         0.66187100446803369657
0.53617502895716315014        0.5598313305769117412
0.6238636460835660344        0.53806262726474883533
0.63526025562783965713         0.60057070070623645773

SyntaxError: invalid syntax (<ipython-input-53-3f96fb624380>, line 2)

In [2]:
GPU Comparison
Inference Time
Text Avg Detection Time : 0.075
Text Avg Recognize Time : 0.034
FCN: 0.318

RAM Needed
SnakeText : 613 Mb
MORAN : 723 Mb
    
FCN : 1806 Mb

SyntaxError: invalid syntax (<ipython-input-2-6885a956a83b>, line 1)

In [28]:
### Text detection result
# single

50
31
3.7401898041236397
0.6701494274416144
50 45 19
0.63331575060660407216

# duplicate
158
112
2.9431014794219026
0.6945257185722118
159 140 46
0.67760461608878017303

# multiple

158
107
3.276136937629111
0.6994557772663099
158 140 51
0.67646705400044632335

# 3
31
17
4.2590200942248435
0.6233106169749204
31 26 14
0.5841011706539726643

# 5
32
22
4.102603415928908
0.625501047702326
32 29 10
0.6096090254814238475

# 7
59
40
2.867427374819038
0.6431336016264122
59 53 19
0.6349789405833061077


0.6773317515173449

In [ ]:
#### Resolution
T: 182
>: 138
theta: 4.410047301369583
av iou>0.5: 0.7641624745124106

180 175 44

0.7762042880533551436

T: 182
>: 150
theta: 4.713590543720187
av iou>0.5: 0.7567561300785954
181 179 32
0.78171875511385041337